# Testing GAN generated data augmentation, using different proportions of data for training GAN 
## This was 

Dataset is COVID_QU_Ex dataset from Kaggle

# Libraries

In [2]:
import os 
import torch
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import random
from PIL import Image
import pickle
import subprocess
import time
import neptune 
import matplotlib
from sklearn.metrics  import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score
import gc
#This code needs a little bit rework, so testing will be easier

In [3]:
ROOT_DIR = 'Lung_Segmentation_Data'
#run = neptune.init_run()

torch.set_num_threads(12)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
gan_directories = {
    'Test_orig_0.8' : {'dir':'2023-03-14_19-00-08','best':'127.0.0.1-5002'},
    'Test_orig_0.6' : {'dir':'2023-03-15_09-51-22','best':'127.0.0.1-5001'},
    'Test_orig_0.4' : {'dir':'2023-03-16_17-23-13','best':'127.0.0.1-5002'},
    'Test_orig_0.2' : {'dir':'2023-03-17_11-28-34','best':'127.0.0.1-5002'},
    
    'Test_0_0.8' : {'dir':'2023-03-23_15-19-22','best':'127.0.0.1-5002'},
    'Test_0_0.6' : {'dir':'2023-03-24_16-23-49','best':'127.0.0.1-5001'},
    'Test_0_0.4' : {'dir':'2023-03-25_09-50-59','best':'127.0.0.1-5001'},
    'Test_0_0.2' : {'dir':'2023-03-25_16-58-45','best':'127.0.0.1-5003'},

    'Test_1_0.8' : {'dir':'2023-03-21_16-13-32','best':'127.0.0.1-5003'},
    'Test_1_0.6' : {'dir':'2023-03-22_09-56-24','best':'127.0.0.1-5001'},
    'Test_1_0.4' : {'dir':'2023-03-22_18-38-27','best':'127.0.0.1-5000'},
    'Test_1_0.2' : {'dir':'2023-03-23_08-16-51','best':'127.0.0.1-5002'},

    'Test_2_0.8' : {'dir':'2023-03-19_18-20-08','best':'127.0.0.1-5001'},
    'Test_2_0.6' : {'dir':'2023-03-20_08-46-21','best':'127.0.0.1-5001'},
    'Test_2_0.4' : {'dir':'2023-03-20_16-58-27','best':'127.0.0.1-5001'},
    'Test_2_0.2' : {'dir':'2023-03-21_08-49-59','best':'127.0.0.1-5002'},

    'Test_3_0.8' : {'dir':'2023-03-17_17-42-29','best':'127.0.0.1-5001'},
    'Test_3_0.6' : {'dir':'2023-03-18_07-56-44','best':'127.0.0.1-5003'},
    'Test_3_0.4' : {'dir':'2023-03-18_23-01-23','best':'127.0.0.1-5002'},
    'Test_3_0.2' : {'dir':'2023-03-19_07-43-42','best':'127.0.0.1-5000'},
     }

# Making datasets

In [5]:

class CustomDataset(torchvision.datasets.ImageFolder):
    """
        A simple imagedatset for storing data
    """
    #Imagefolder for efficiency
    def __init__(self, images, transform):
        target_dir = os.path.join(ROOT_DIR, 'original')
        super().__init__(target_dir, transform)
        self.samples = images
        self.imgs = images

def DatasetMaker(split, mode=None, data_ratio=1, transform = None, geoaugment=False, random_affine = 0, seed = 0):
    """
        Returns a CustomDataset with given parameters
        split: str, determines train-test to use; options: 'orig', '0', '1', '2', '3'
        mode: str, 'oversampling', 'gan' or None
            'oversampling' : oversample with real images to balance classes
                    'gan' : balance datasets with gan generated images (uses data_ratio to figure out which gan to use)
                    None  : dataset won't be balanced
        data_ratio (optional): int, the ratio of the training covid data to be used
                    options:
                        '1' : all data
                        '0.8' : 80% of training images
                        '0.6' : 60% of training images
                        '0.4' : 40% of training images
                        '0.2' : 20% of training images
        transform (optional): torch.Compose instance, sets the dataset's transforms
        geougment (optional): bool, uses basic data augmentation techmiques
        seed (optional): int, seed to use for reproducibility
    """
    #This code prepares my fixed set of images to replace the Imagefolder's original images
    #Maybe this could be done nicer with overwriting the DatasetFolder's find_classes method, but currently this works
    # the idea is to make (route, index) pairs for later loading in images
    classes = ['normal', 'viral', 'covid']

    #Determining the root directories for original images 
    orig_dirs = {
    'normal' : f'{ROOT_DIR}/original/Normal',
    'viral' : f'{ROOT_DIR}/original/Non-Covid',
    'covid' : f'{ROOT_DIR}/original/COVID-19'
    }
    #Determining the root directories for generated images 
    fake_dirs = {
        'gan_0.8' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.8',
        'gan_0.6' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.6',
        'gan_0.4' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.4',
        'gan_0.2' : f'{ROOT_DIR}/generated/Test_{split}/gan_0.2'
    }
    #Determining the root directories for files which contain the names of the COVID19 pictures 
    indicies_files = {
        'gan_0.8' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.8_gan.pkl', 
        'gan_0.6' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.6_gan.pkl',
        'gan_0.4' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.4_gan.pkl',
        'gan_0.2' : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_0.2_gan.pkl',
        'test'  : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_test.pkl',
        'train'  : f'{ROOT_DIR}/Indicies_files/Test_{split}/{split}_split_train_and_val.pkl'
    }
    class_idx = {
        'covid': 0, 
        'viral': 1,
        'normal': 2,
        'gan_0.8' : 0,
        'gan_0.6' : 0,
        'gan_0.4' : 0,
        'gan_0.2' : 0
    }
    idx_to_class ={
        0: 'covid',
        1: 'viral',
        2: 'normal'
    }
    valid_ratios = [1, 0.8, 0.6, 0.4, 0.2]

    #The dictionary that will contain the route for the several image classes
    source_dir = {}
    train_images = []

    #creating the sources for the classes
    for class_name in classes: 
        source_dir[class_name] = orig_dirs[class_name]
    
    #Get all training images
    file = indicies_files['train'] 
    imgs = load_images_from_file(file)
    imgs = [*imgs[0],*imgs[1]] #The file contains (train, val) sets
    
    #Making the paths for the training images
    num_of_covid_imgs = 0 #?
    for x in imgs: 
        class_of_x = idx_to_class[x[1]] #The images are saved in (image_name, class_index) format
        item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] 
        if item[1] == class_idx['covid']: num_of_covid_imgs +=1 #?
        train_images.append(item)
    #If data_ratio is not 1, we need to change the covid images of the dataset
    if data_ratio!=1:
        num_of_covid_imgs = 0
        train_images = [x for x in train_images if x[1]!=class_idx['covid']]
        file = indicies_files[f'gan_{data_ratio}']
        imgs = load_images_from_file(file)
        for x in imgs:
            class_of_x = idx_to_class[x[1]]
            item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] 
            if item[1] == class_idx['covid']: num_of_covid_imgs +=1 
            train_images.append(item)
    
    #If picture generation is needed we want to know the average class size and missing number of covid images
    average_class_size = (len(train_images)-num_of_covid_imgs)//2
    missing_images = max(0, average_class_size - num_of_covid_imgs)

    #Determining wether image generation is needed and which kind of it then making the generation
    if mode=='gan' and (data_ratio in valid_ratios):
        gan = f'gan_{data_ratio}'
        gan_dir = fake_dirs[gan]
        #generate_images_to_dir(split, data_ratio, gan, gan_dir, missing_images) #?
        gan_ims = []
        for x in os.listdir(gan_dir): #optimize further
            if x.lower().endswith('jpg'):
                item = os.path.join(gan_dir, x), class_idx['covid']
                gan_ims.append(item)
        sample = random.sample(gan_ims, missing_images)
        train_images = [*train_images,*sample]
    elif mode=='oversampling':
        covid_images = [x for x in train_images if x[1]==class_idx['covid']]
        batch_size = len(covid_images)
        while batch_size <= missing_images:
            train_images = [*train_images, *covid_images]
            missing_images -= batch_size
        if missing_images>0:
            sample = random.sample(covid_images, missing_images)
            train_images = [*train_images, *sample]
    
    #Making the paths for the test images
    test_images = []
    test_imgs = load_images_from_file(indicies_files['test'])
    for x in test_imgs:
        class_of_x = idx_to_class[x[1]]
        item = os.path.join(source_dir[class_of_x],x[0]),class_idx[class_of_x] #This should be correct
        test_images.append(item)
    random.Random(seed).shuffle(test_images)
    bound = len(test_images)//2
    val_images = test_images[bound:]
    test_images = test_images[:bound]
    #Making some basic transforms 
    if transform is None:
        transforms = [torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    torchvision.transforms.Grayscale(num_output_channels=1)]
    else:
        transforms = transform

    #Checking wether geometric augmentation is needed ( classic augmentation methods)
    if geoaugment:
        augmentation_transforms = [torchvision.transforms.RandomAffine(random_affine),
                            torchvision.transforms.Resize(size=(140,140)),
                                   #torchvision.transforms.RandomCrop(size = (128,128))]
                            torchvision.transforms.FiveCrop(size = (128,128)),
                            torchvision.transforms.Lambda(lambda crops: torch.stack([crop for crop in crops]))]
        transforms = [*transforms,*augmentation_transforms]
    else:
        size_transform = [torchvision.transforms.Resize(size=(128, 128))]
        transforms = [*transforms,*size_transform]
    transforms = torchvision.transforms.Compose(transforms)
    train_dataset = CustomDataset(train_images, transforms)
    val_dataset = CustomDataset(val_images, transforms)
    test_dataset = CustomDataset(test_images,  transforms)
    return train_dataset, val_dataset, test_dataset

def load_images_from_file(file):
    with open(file, 'rb') as file:
        data = pickle.load(file)
    return data   
    
#def generate_images_to_dir(split, data_ratio, gan, directory, size):
#    """
#        Generates pictures with a given gan, to a given directory
#    """
#    #Goes into Lipizzaner's directory and then generates images with a given GAN (this function is a Lipizzaner built-in method)
#    #Then returns into this directory
#    
#    curr_dir = os.getcwd()
#    print(curr_dir)
#    
#    lippi_dir = '/home/bbernard/lipizzaner-covidgan-master/src/'  #Change this on server
#    
#    output_dir = os.path.join(curr_dir, directory) #?
#   
#   #Gan to use is determined by the split and the data_ratio parameters
#    gan_dir = gan_directories[f'Test_{split}_{data_ratio}'] 
#    src_dir = os.path.join(lippi_dir, f'output/lipizzaner_gan/master/{gan_dir}/127.0.0.1-5000')
#    
#    config_file = os.path.join(lippi_dir, f'configuration/covid-qu-conv/Test_{split}/covidqu_{data_ratio}.yml')
#    
#    #man = os.path.join(lippi_dir, 'main.py')#
#
#    code =f'python main.py generate --mixture-source {src_dir} -o {output_dir} --sample-size {size} -f {config_file}'
#    os.chdir(lippi_dir)
#    subprocess.run(code, shell=True)
#    os.chdir(curr_dir)

## Models


In [6]:
def get_model(name):
    """
        Returns pretrained models
    """
    if name=="resnet":
        resnet18 = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        resnet18.conv1= torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = torch.nn.Linear(in_features=512, out_features=3)
        resnet18.get_name = 'resnet18'
        return resnet18
    elif name=="vgg":
        vgg16 = torchvision.models.vgg16(weights = torchvision.models.VGG16_Weights.IMAGENET1K_V1)
        vgg16.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        vgg16.classifier[6] = torch.nn.Linear(in_features=4096, out_features=3, bias=True)
        vgg16.get_name = 'vgg16'
        return vgg16
    elif name=="efficient":
        efficient = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        efficient.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        efficient.classifier[1] = torch.nn.Linear(in_features=1280, out_features=3, bias=True)
        efficient.get_name = 'efficientnet_b0'
        return efficient
    else:
        print("Not implemented")

## Train function

In [7]:

def train(epochs, net, loss_fn, optimizer, weight_decay, train_dataset, test_dataset, batch_size, shuffle, neptune_run): #, neptune_run ):
    """
        A simple train function 
        Params: 
            epoch: number of epochs to train for
            modell: The neural network to train
            loss_fn: Loss function instance
            optimizer: optimizer instance
            train_dataset (Dataset)
            test_dataset (Dataset)
            batch_size (int)
            shuffle (bool) 
    """
    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    test_dl =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers=0, shuffle= shuffle)
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=weight_decay)
    print('Starting training..')
    net.to(device)
    for e in range(epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_iter = iter(train_dl)
        
        train_accuracy = 0.
        train_loss = 0.
        
        sample_num = 0 

        net.train() #set model to training phase
        
        #Training 
        batch_num = 0
        
        while batch_num< len(train_dl):
            images, labels = next(train_iter)
            if len(images.size())==5:
                bs, ncrops, c, h, w = images.size()
            else:
                bs, c, h, w = images.size()
                ncrops = 1
            images = images.view(-1, c,h,w)
            images = images.to(device) 
            
            labels = labels.to(device)
            #labels = labels.repeat_interleave(ncrops)
            
            outputs = net(images)
            outputs = outputs.view(bs, ncrops, -1).mean(1) 
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #train_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            
            sample_num += len(labels)
            train_accuracy += sum((preds == labels).cpu().numpy())
            train_loss += loss.item()
            batch_num += 1
        
        train_loss/= len(train_dl)
        train_accuracy/= len(train_dataset)
        print(f'Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')
        neptune_run['train/train_loss'].append(train_loss)
        neptune_run['train/train_accuracy'].append(train_accuracy)
        
        del train_loss
        del train_accuracy
        del images
        del labels
        del outputs
        del loss
        gc.collect()
        torch.cuda.empty_cache() 
        
        val_loss = 0.
        val_accuracy = 0.

        val_batch_num = 0
        val_iter = iter(test_dl)
        y_true = []
        y_pred = []

        net.eval()
        while val_batch_num < len(val_iter):
            images, labels = next(val_iter)
            if len(images.size())==5:
                bs, ncrops, c, h, w = images.size()
            else:
                bs, c, h, w = images.size()
                ncrops = 1
            
            images = images.view(-1, c,h,w)
            images = images.to(device) 
            
            #labels = labels.repeat_interleave(ncrops)
            y_true.extend(labels)
            labels = labels.to(device)
            
            outputs = net(images)
            outputs = outputs.view(bs, ncrops, -1).mean(1) 
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.detach().cpu())
            val_accuracy += sum((preds == labels).cpu().numpy())
            val_batch_num += 1

        val_loss /= len(test_dl)
        val_accuracy = val_accuracy/len(test_dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
        scheduler.step()
        net.train()

        neptune_run['train/val_loss'].append(val_loss)
        neptune_run['train/val_accuracy'].append(val_accuracy)

        curr_conf_matrix = confusion_matrix(y_true, y_pred) 
        curr_conf_matrix = curr_conf_matrix / np.sum(curr_conf_matrix)
        im = ConfusionMatrixDisplay(curr_conf_matrix, display_labels=["covid", "viral","normal"]).plot()
        neptune_run['metrics/conf_matrix'].append(im.figure_) #, description=f"Confusion matrix in the iteration: {iteration}"  File.as_image(curr_conf_matrix))
        matplotlib.pyplot.close()
        neptune_run['metrics/acc'].append( accuracy_score(y_true, y_pred))
        neptune_run['metrics/bal_acc'].append(balanced_accuracy_score(y_true, y_pred))
        neptune_run['metrics/recall'].append(recall_score(y_true, y_pred,average ="micro"))
        neptune_run['metrics/precision'].append( precision_score(y_true, y_pred, average ="micro"))
        neptune_run['metrics/f1'].append( f1_score(y_true, y_pred,average ="micro"))    
        #neptune_run['val/loss'].append(val_loss)
        #neptune_run['val/accuracy'].append(val_accuracy)
        del val_loss
        del val_accuracy
        del images
        del labels
        del outputs
        del loss
        del y_pred
        del y_true
        gc.collect()
        torch.cuda.empty_cache() 
        
    print('Training complete..')
    return net


In [8]:
def save_history(network,folder, case_string): #,history
    """
        A simple function that saves the histories and the clasificator net
    """
    FILEBASE = f"Histories/{folder}/{case_string}"
    torch.save(network.state_dict(), FILEBASE + '.pt')
    #with open(FILEBASE + '-history.pkl', 'wb') as file:
    #    pickle.dump(history, file)
    #    print(f'{FILEBASE} instance saved')


In [9]:
gc.collect()
torch.cuda.empty_cache()

#### Hyperparameters and debugging


In [ ]:
from collections import defaultdict #neptune_run = defaultdict(list) #
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'id': time.strftime("%H-%M-%S"),
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 1, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = defaultdict(list) #neptune.init_run() 
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
#neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{params['id']}"
save_history(net,params['network_name'], case_string)


Starting training..
Starting epoch 1/1


### Longer runs


In [ ]:
#VGG runs
#1  1
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#2 1
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#3   0.8
params = {
'split' : 'orig',
'data_ratio' : 0.8,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#4
params = {
'split' : 'orig',
'data_ratio' : 0.8,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

##################
#5 0.6
params = {
'split' : 'orig',
'data_ratio' : 0.6,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#6
params = {
'split' : 'orig',
'data_ratio' : 0.6,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

##################
#7 0.4
params = {
'split' : 'orig',
'data_ratio' : 0.4,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#8
params = {
'split' : 'orig',
'data_ratio' : 0.4,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#############################
#9 0.2
params = {
'split' : 'orig',
'data_ratio' : 0.2,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#10 0.2
params = {
'split' : 'orig',
'data_ratio' : 0.2,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

/tmp/ipykernel_1722917/2293176850.py:25: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run()


https://app.neptune.ai/aielte/CovidGAN/e/COV1-286
Starting training..
Starting epoch 1/30
Training Loss: 0.4542, Accuracy: 0.8171
Validation Loss: 0.2498, Accuracy: 0.9095
Starting epoch 2/30
Training Loss: 0.2687, Accuracy: 0.8955
Validation Loss: 0.2177, Accuracy: 0.9293
Starting epoch 3/30
Training Loss: 0.2207, Accuracy: 0.9158
Validation Loss: 0.1620, Accuracy: 0.9440
Starting epoch 4/30
Training Loss: 0.1828, Accuracy: 0.9294
Validation Loss: 0.1757, Accuracy: 0.9405
Starting epoch 5/30
Training Loss: 0.1584, Accuracy: 0.9395
Validation Loss: 0.1767, Accuracy: 0.9349
Starting epoch 6/30
Training Loss: 0.0831, Accuracy: 0.9686
Validation Loss: 0.1331, Accuracy: 0.9543
Starting epoch 7/30
Training Loss: 0.0577, Accuracy: 0.9789
Validation Loss: 0.1560, Accuracy: 0.9561
Starting epoch 8/30
Training Loss: 0.0364, Accuracy: 0.9874
Validation Loss: 0.2104, Accuracy: 0.9493
Starting epoch 9/30
Training Loss: 0.0192, Accuracy: 0.9936
Validation Loss: 0.2571, Accuracy: 0.9461
Starting epo

Validation Loss: 0.2592, Accuracy: 0.9549
Starting epoch 25/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2598, Accuracy: 0.9546
Starting epoch 26/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2593, Accuracy: 0.9546
Starting epoch 27/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2594, Accuracy: 0.9546
Starting epoch 28/30
Training Loss: 0.0012, Accuracy: 1.0000
Validation Loss: 0.2593, Accuracy: 0.9546
Starting epoch 29/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2593, Accuracy: 0.9546
Starting epoch 30/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2593, Accuracy: 0.9546
Training complete..
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-287/metadata
https://app.neptune.

Starting epoch 16/30
Training Loss: 0.0012, Accuracy: 0.9999
Validation Loss: 0.2840, Accuracy: 0.9517
Starting epoch 17/30
Training Loss: 0.0011, Accuracy: 0.9999
Validation Loss: 0.2849, Accuracy: 0.9520
Starting epoch 18/30
Training Loss: 0.0011, Accuracy: 1.0000
Validation Loss: 0.2863, Accuracy: 0.9517
Starting epoch 19/30
Training Loss: 0.0013, Accuracy: 0.9999
Validation Loss: 0.2871, Accuracy: 0.9517
Starting epoch 20/30
Training Loss: 0.0011, Accuracy: 0.9999
Validation Loss: 0.2883, Accuracy: 0.9526
Starting epoch 21/30
Training Loss: 0.0010, Accuracy: 0.9999
Validation Loss: 0.2883, Accuracy: 0.9526
Starting epoch 22/30
Training Loss: 0.0010, Accuracy: 0.9999
Validation Loss: 0.2883, Accuracy: 0.9526
Starting epoch 23/30
Training Loss: 0.0011, Accuracy: 0.9999
Validation Loss: 0.2884, Accuracy: 0.9523
Starting epoch 24/30
Training Loss: 0.0011, Accuracy: 0.9999
Validation Loss: 0.2885, Accuracy: 0.9523
Starting epoch 25/30
Training Loss: 0.0010, Accuracy: 1.0000
Validation L

Training Loss: 0.0376, Accuracy: 0.9871
Validation Loss: 0.2385, Accuracy: 0.9402
Starting epoch 8/30
Training Loss: 0.0241, Accuracy: 0.9925
Validation Loss: 0.2572, Accuracy: 0.9414
Starting epoch 9/30
Training Loss: 0.0141, Accuracy: 0.9965
Validation Loss: 0.2747, Accuracy: 0.9411
Starting epoch 10/30
Training Loss: 0.0087, Accuracy: 0.9979
Validation Loss: 0.3110, Accuracy: 0.9420
Starting epoch 11/30
Training Loss: 0.0037, Accuracy: 0.9997
Validation Loss: 0.3166, Accuracy: 0.9431
Starting epoch 12/30
Training Loss: 0.0032, Accuracy: 0.9997
Validation Loss: 0.3241, Accuracy: 0.9425
Starting epoch 13/30
Training Loss: 0.0028, Accuracy: 0.9997
Validation Loss: 0.3294, Accuracy: 0.9428
Starting epoch 14/30
Training Loss: 0.0025, Accuracy: 0.9998
Validation Loss: 0.3368, Accuracy: 0.9423
Starting epoch 15/30
Training Loss: 0.0021, Accuracy: 0.9998
Validation Loss: 0.3654, Accuracy: 0.9408
Starting epoch 16/30
Training Loss: 0.0017, Accuracy: 0.9999
Validation Loss: 0.3440, Accuracy: 

Done!
Waiting for the remaining 8 operations to synchronize with Neptune. Do not kill this process.
All 8 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-292/metadata
https://app.neptune.ai/aielte/CovidGAN/e/COV1-293
Starting training..
Starting epoch 1/30
Training Loss: 0.5619, Accuracy: 0.7661
Validation Loss: 0.3063, Accuracy: 0.8872
Starting epoch 2/30
Training Loss: 0.3329, Accuracy: 0.8664
Validation Loss: 0.3117, Accuracy: 0.8836
Starting epoch 3/30
Training Loss: 0.2593, Accuracy: 0.8976
Validation Loss: 0.2102, Accuracy: 0.9246
Starting epoch 4/30
Training Loss: 0.2163, Accuracy: 0.9151
Validation Loss: 0.3043, Accuracy: 0.8966
Starting epoch 5/30
Training Loss: 0.1731, Accuracy: 0.9320
Validation Loss: 0.2772, Accuracy: 0.9172
Starting epoch 6/30
Training Loss: 0.0800, Accuracy: 0.9704
Validation Loss: 0.2275, Accuracy: 0.9346
Starting epoch 7/30
Training Loss: 0.0562, Accuracy: 0.9799
Validation Lo

In [ ]:
#VGG runs
#1  1_True
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.2,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#2 1_True
params = {
'split' : 'orig',
'data_ratio' : 1,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'weight_decay':0.2,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#3   0.8_True
params = {
'split' : 'orig',
'data_ratio' : 0.8,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#4
params = {
'split' : 'orig',
'data_ratio' : 0.8,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

##################
#5 0.6_True
params = {
'split' : 'orig',
'data_ratio' : 0.6,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#6
params = {
'split' : 'orig',
'data_ratio' : 0.6,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

##################
#7 0.4_True
params = {
'split' : 'orig',
'data_ratio' : 0.4,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)


#8
params = {
'split' : 'orig',
'data_ratio' : 0.4,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': False,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 12e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#############################
#9 0.2_True
params = {
'split' : 'orig',
'data_ratio' : 0.2,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 9e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

#10 0.2
params = {
'split' : 'orig',
'data_ratio' : 0.2,   #1, 0.8, 0.6, 0.4, 0.2
'augment' : 'None',       #gan, oversampling, None
'geoaugment': True,
'network_name' : "vgg", #resnet, vgg, efficient
'loss_function' : "CrossEntropyLoss",
'optimizer' : "adam",
'batch_size' : 32,
'epochs' : 30, #30/40
'adam_lr' : 3e-5,
'weight_decay':0.1,
'degree':3
}

network = get_model(params['network_name'])
optimizer = torch.optim.Adam(network.parameters(), lr=params['adam_lr'])
loss_fn = torch.nn.CrossEntropyLoss()

train_dataset, val_dataset, _ = DatasetMaker(split = params['split'],mode = params['augment'], 
                                data_ratio = params['data_ratio'],geoaugment = params['geoaugment'],random_affine = params['degree'])

neptune_run = neptune.init_run() 
run_id = neptune_run["sys/id"].fetch()
neptune_run['algorithm'] = "Classifier"
neptune_run['params'] = params
net = train(params['epochs'], network, loss_fn, optimizer, params['weight_decay'],
                train_dataset, val_dataset, params['batch_size'],shuffle = True, neptune_run = neptune_run)
neptune_run.stop()

case_string = f"{params['network_name']}_{params['split']}_{params['data_ratio']}_{params['augment']}_{params['geoaugment']}_{run_id}"
save_history(net, params['network_name'], case_string)

/tmp/ipykernel_1770571/2692008613.py:25: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  neptune_run = neptune.init_run()


https://app.neptune.ai/aielte/CovidGAN/e/COV1-297
Starting training..
Starting epoch 1/30
Training Loss: 0.0888, Accuracy: 0.1643
Validation Loss: 0.0624, Accuracy: 0.1800
Starting epoch 2/30
Run COV1-297 received abort signal. Exiting
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/aielte/CovidGAN/e/COV1-297/metadata
